In [ ]:
# 서울시 홈페이지 
1 서울 소식 게시글 
2.1~5p 
3 게시글 제목 / url 추출 
4 url 이동 후 당당부서, 업로드 날짜
5 부서별 업로드 현황 
6 날짜별 업로드 현황

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import time
import pandas as pd 
from urllib.parse import urljoin # 상대경로일 경우 url을 합쳐서 완성해주는 라이브러리/ 안전하게 항상 사용하는걸 추천



service = Service(ChromeDriverManager().install())
# instence(붕어빵 실질적 결과값) / class = 붕어빵 틀 

options =  Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920X1080")
options.add_argument("--start_maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 목록 제목& 링크값을 수집하는 함수 선언
def collect_list_links_on_current_page() : 
    base = driver.current_url # https://www.seoul.go.kr/realmnews/in/list.do
    items = driver.find_elements(By.CSS_SELECTOR, "div.news-lst div.item > a")
    
    links =[] 
    for item in items : 
        title = item.find_element(By.CSS_SELECTOR, "em.subject").text.strip()
        href = item.get_attribute("href") # https://news.seoul.go.kr/gov/archives/574513"
        url = urljoin(base, href)
        links.append({"list_title" : title, "url" : url})
    return links

# 각 페이지당 부서명, 날짜 함수 선언 
def scape_detail_fields(article_url) :
    driver.get(article_url)

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#view_top")))

    title = driver.find_element(By.CSS_SELECTOR, "#view_top h3").text.strip()
    
    dept_spans = driver.find_elements(By.CSS_SELECTOR, "#view_top dd.dept span")
    # [일자리정책과, 고용훈련팀]
    dept = "-".join([s.text.strip() for s in dept_spans if s.text.strip()]) #join: 각각의 값을 연결  ex. 일자리정책과-고용훈련팀
    modified_date = driver.find_element(By.CSS_SELECTOR, "#view_top dd.date").text.strip()
    return title, dept, modified_date

try : 
    driver.get("https://www.seoul.go.kr/main/index.jsp")
    time.sleep(2)

    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='/realmnews/in/list.do']"))). click()

    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.news-lst div.item > a")))  # 함수를 호출하는 지점의 url 

    max_page = 5
    collected_links = []
    
    for page in range(1,  max_page + 1) : 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)

        # 함수 호출 지점
        collected_links.extend(collect_list_links_on_current_page())

        print(f"[목록수집] {page}페이지 누적 링크 {len(collected_links)}개")

        if page == max_page : 
            break

        next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#paging_main ul.pagination a[data-page='{page + 1}']")))
        next_btn.click()

        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.3)

    tmp_df = pd.DataFrame(collected_links).drop_duplicates(subset=["url"]).reset_index(drop=True)
    link_list = tmp_df["url"].to_list()


    rows = []
    for i, link in enumerate(link_list, start=1) : 
        try: 
            title, dept, modified_date = scape_detail_fields(link)
            rows.append({
                "title": title,
                "url": link,
                "dept": dept, 
                "modified_date": modified_date
            })
            print (f"[상세수집] {i}/{len(link_list)} 완료")
        except Exception as e: 
            rows.append({
              "title": "",
              "url": link,
              "dept": "", 
              "modified_date": "",
              "error": str(e)
            })
            print(f"[상세실패] {i}/{len(link_list)} {link} -> {e}")

        time. sleep(0.5)
    df = pd.DataFrame(rows)
    filename = "seoul_news_p1_to_p5 with_dept_date.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    
finally : 
   driver.quit()
    



[목록수집] 1페이지 누적 링크 10개
[목록수집] 2페이지 누적 링크 20개
[목록수집] 3페이지 누적 링크 30개
[목록수집] 4페이지 누적 링크 40개
[목록수집] 5페이지 누적 링크 50개
[상세수집] 1/50 완료
[상세수집] 2/50 완료
[상세수집] 3/50 완료
[상세수집] 4/50 완료
[상세수집] 5/50 완료
[상세수집] 6/50 완료
[상세수집] 7/50 완료
[상세수집] 8/50 완료
[상세수집] 9/50 완료
[상세수집] 10/50 완료
[상세수집] 11/50 완료
[상세수집] 12/50 완료
[상세수집] 13/50 완료
[상세수집] 14/50 완료
[상세수집] 15/50 완료
[상세수집] 16/50 완료
[상세수집] 17/50 완료
[상세수집] 18/50 완료
[상세수집] 19/50 완료
[상세수집] 20/50 완료
[상세수집] 21/50 완료
[상세수집] 22/50 완료
[상세수집] 23/50 완료
[상세수집] 24/50 완료
[상세수집] 25/50 완료
[상세수집] 26/50 완료
[상세수집] 27/50 완료
[상세수집] 28/50 완료
[상세수집] 29/50 완료
[상세수집] 30/50 완료
[상세수집] 31/50 완료
[상세수집] 32/50 완료
[상세수집] 33/50 완료
[상세수집] 34/50 완료
[상세수집] 35/50 완료
[상세수집] 36/50 완료
[상세수집] 37/50 완료
[상세수집] 38/50 완료
[상세수집] 39/50 완료
[상세수집] 40/50 완료
[상세수집] 41/50 완료
[상세수집] 42/50 완료
[상세수집] 43/50 완료
[상세수집] 44/50 완료
[상세수집] 45/50 완료
[상세수집] 46/50 완료
[상세수집] 47/50 완료
[상세수집] 48/50 완료
[상세수집] 49/50 완료
[상세수집] 50/50 완료


In [ ]:
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")

In [ ]:
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")